In [5]:
!pip install tweepy

In [13]:
import tweepy
import pandas as pd

# Your Twitter Developer credentials
API_KEY = "z6CyDkO5IoWRHWx8pKTCUi3Y6"
API_SECRET = "k2foShbN4ieRUkFlzC2A5YhG4Q1SK6eeCwt9VR669R3843xo30"
ACCESS_TOKEN = "2365212734-by5FTYejUqlppZpqu047Cf7ptE10BLusIogMR0K"
ACCESS_TOKEN_SECRET = "7jqRUdAynwMHd76Ouj3bdvPf7yhFmWrw3eC4vmr6bF6Qn"

# Authenticate with Twitter
auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

print("✅ Twitter API authenticated!")


✅ Twitter API authenticated!


In [19]:
import tweepy
import pandas as pd

# v2 client setup (use Bearer Token from Twitter Developer portal)
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAHX23AEAAAAAYz%2FOOgOuvxOGhV60eMiEfm632aw%3DVYN6ug2x5gn2AfMyZLdCbkXzbH4YKGsI3bSWxS6aA9Miainu1g"

client = tweepy.Client(bearer_token=BEARER_TOKEN)

query = "India Elections -is:retweet lang:en"
max_results = 50  # limit for free tier per call

response = client.search_recent_tweets(query=query,
                                       tweet_fields=["created_at", "author_id", "text"],
                                       max_results=max_results)

data = []
for tweet in response.data:
    data.append([tweet.id, tweet.created_at, tweet.author_id, tweet.text])

df = pd.DataFrame(data, columns=["Tweet_ID", "Timestamp", "Author_ID", "Text"])
df.head()


,Tweet_ID,Timestamp,Author_ID,Text
0,1948730831612137695,2025-07-25 13:03:30+00:00,602691307,@ECISVEEP @airnewsalerts @DDNewslive @DDNewsHi...
1,1948730160091455957,2025-07-25 13:00:50+00:00,865470149763059712,"Hey @RahulGandhi , Election commission is com..."
2,1948729842033119259,2025-07-25 12:59:34+00:00,1833711472087572480,@PMRCLofficial @AbhaySinghIAS @_sharma999 @IPR...
3,1948728519351030077,2025-07-25 12:54:19+00:00,1661981214385872896,@ugiboogie2 @sandeep_PT This govt. has to fall...
4,1948727888142077972,2025-07-25 12:51:48+00:00,1869363800710807552,@MeghUpdates I don't understand why congress k...


In [21]:
import re
import string

def clean_tweet(text):
    text = re.sub(r'http\S+', '', text)                    # Remove URLs
    text = re.sub(r'@\w+', '', text)                       # Remove @mentions
    text = re.sub(r'#', '', text)                          # Remove hashtag symbol
    text = re.sub(r'[^\x00-\x7F]+', '', text)              # Remove emojis/non-ASCII
    text = re.sub(r'\d+', '', text)                        # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.lower().strip()                            # Lowercase and trim
    return text

# Apply to DataFrame
df["Clean_Text"] = df["Text"].apply(clean_tweet)
df[["Text", "Clean_Text"]].head()


,Text,Clean_Text
0,@ECISVEEP @airnewsalerts @DDNewslive @DDNewsHi...,greatly welcome none has the authority to diff...
1,"Hey @RahulGandhi , Election commission is com...",hey election commission is committed to ind...
2,@PMRCLofficial @AbhaySinghIAS @_sharma999 @IPR...,please dont rush work due to elections politic...
3,@ugiboogie2 @sandeep_PT This govt. has to fall...,this govt has to fall in a matter of timeso th...
4,@MeghUpdates I don't understand why congress k...,i dont understand why congress keeps repeating...


In [27]:
!pip install textblob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 604.5 kB/s eta 0:00:00a 0:00:01


In [29]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/santhoshms/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [31]:
from textblob import TextBlob

def get_polarity(text):
    return TextBlob(text).sentiment.polarity

def get_sentiment_label(polarity):
    if polarity > 0.1:
        return "Positive"
    elif polarity < -0.1:
        return "Negative"
    else:
        return "Neutral"

df["Polarity"] = df["Clean_Text"].apply(get_polarity)
df["Sentiment"] = df["Polarity"].apply(get_sentiment_label)

df[["Clean_Text", "Polarity", "Sentiment"]].head()


,Clean_Text,Polarity,Sentiment
0,greatly welcome none has the authority to diff...,0.390000,Positive
1,hey election commission is committed to ind...,0.033333,Neutral
2,please dont rush work due to elections politic...,0.051623,Neutral
3,this govt has to fall in a matter of timeso th...,-0.100000,Neutral
4,i dont understand why congress keeps repeating...,-0.056667,Neutral


In [33]:
df.to_csv("tweets.csv", index=False)
print("✅ tweets.csv saved successfully!")


✅ tweets.csv saved successfully!


In [39]:
import re

# Function to extract hashtags
def extract_hashtags(text):
    return re.findall(r"#(\w+)", str(text))  # Extract words after #

# Apply to raw tweet text column
df["Hashtags"] = df["Text"].apply(extract_hashtags)

# Show example
df[["Text", "Hashtags"]].head()


,Text,Hashtags
0,@ECISVEEP @airnewsalerts @DDNewslive @DDNewsHi...,[]
1,"Hey @RahulGandhi , Election commission is com...",[]
2,@PMRCLofficial @AbhaySinghIAS @_sharma999 @IPR...,[]
3,@ugiboogie2 @sandeep_PT This govt. has to fall...,[]
4,@MeghUpdates I don't understand why congress k...,[]


In [41]:
# Expand hashtags into separate rows
hashtags_df = df.explode("Hashtags").dropna(subset=["Hashtags"])

hashtags_df[["Hashtags", "Sentiment"]].head()


,Hashtags,Sentiment
7,VoterRights,Positive
7,Bihar,Positive
8,Sanghis,Positive
8,ResistIndia,Positive
15,cpimspeak,Neutral


In [43]:
# Count top hashtags by sentiment
sentiment_hashtags = (
    hashtags_df.groupby(["Sentiment", "Hashtags"])
    .size()
    .reset_index(name="Count")
    .sort_values(["Sentiment", "Count"], ascending=[True, False])
)

sentiment_hashtags.head(10)


,Sentiment,Hashtags,Count
0,Neutral,CPIM,1
1,Neutral,FakeEncounters,1
2,Neutral,FalseNarrative,1
3,Neutral,IndiaElections2025,1
4,Neutral,Islamophobia,1
5,Neutral,ModiExposed,1
6,Neutral,cpimspeak,1
7,Positive,Bihar,1
8,Positive,ECI,1
9,Positive,ResistIndia,1


In [45]:
# Top 5 for each sentiment
for sentiment in ["Positive", "Neutral", "Negative"]:
    print(f"\nTop hashtags for {sentiment}:")
    print(sentiment_hashtags[sentiment_hashtags["Sentiment"] == sentiment].head(5))



Top hashtags for Positive:
   Sentiment     Hashtags  Count
7   Positive        Bihar      1
8   Positive          ECI      1
9   Positive  ResistIndia      1
10  Positive      Sanghis      1
11  Positive  VoterRights      1

Top hashtags for Neutral:
  Sentiment            Hashtags  Count
0   Neutral                CPIM      1
1   Neutral      FakeEncounters      1
2   Neutral      FalseNarrative      1
3   Neutral  IndiaElections2025      1
4   Neutral        Islamophobia      1

Top hashtags for Negative:
Empty DataFrame
Columns: [Sentiment, Hashtags, Count]
Index: []


In [49]:
import re

# Extract hashtags
def extract_hashtags(text):
    return re.findall(r"#(\w+)", str(text))

df["Hashtags"] = df["Text"].apply(extract_hashtags)

# Expand hashtags to rows
hashtags_df = df.explode("Hashtags").dropna(subset=["Hashtags"])

# Group and count hashtags by sentiment
sentiment_hashtags = (
    hashtags_df.groupby(["Sentiment", "Hashtags"])
    .size()
    .reset_index(name="Count")
    .sort_values(["Sentiment", "Count"], ascending=[True, False])
)

# Save to CSV
sentiment_hashtags.to_csv("hashtags_by_sentiment.csv", index=False)
print("✅ Saved as hashtags_by_sentiment.csv")


✅ Saved as hashtags_by_sentiment.csv
